In [1]:
# Verify gpu info
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Dec  9 17:28:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   51C    P8    N/A /  N/A |      0MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from huggingface_hub import notebook_login

notebook_login()

## Load Dataset

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()
common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "zh-CN", split="test", use_auth_token=True)
print(common_voice)

In [ ]:
# Save dataset to google drive
import os

dataset_dir = "G:\我的云端硬盘\whisper\ASR_data"
os.makedirs(dataset_dir, exist_ok = True)

common_voice.save_to_disk(dataset_dir)

## Feature engineering

In [ ]:
# Retrieve common voice from google drive
from datasets import load_dataset, DatasetDict

dataset_dir = 'G:\我的云端硬盘\whisper\ASR_data'
common_voice = DatasetDict.load_from_disk(dataset_dir)

print(common_voice)

In [3]:
# Process data
from feature_pipline import *

feature_pip = feature_pip(common_voice)
common_voice_processed = feature_pip.pipline()

In [ ]:
# Save the processed data to google drive
import os

data_dir = 'G:\我的云端硬盘\whisper\processed_data'
os.makedirs(data_dir, exist_ok=True)

common_voice_processed.save_to_disk(data_dir)

## Training and Evaluation

In [ ]:
# Retrieve processed data from google drive
from datasets import load_dataset, DatasetDict

dataset_dir = 'G:\我的云端硬盘\whisper\processed_data'
common_voice_processed = DatasetDict.load_from_disk(dataset_dir)
print(common_voice_processed)

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 39637
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 10581
    })
})


In [4]:
# Train a model and upload it to hugging face
from training_pipline import *

learning_rate = 1e-5
warmup_steps=250
max_steps=2000
save_steps=500
eval_steps=500

trainer = Trainer(common_voice_processed, learning_rate, warmup_steps, max_steps, save_steps, eval_steps)
trainer.train()